In [61]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [63]:
path = "../training/conf/network/convnext.yaml"

In [64]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [65]:
cfg

{'_target_': 'text_recognizer.networks.convnext.ConvNext', 'dim': 16, 'dim_mults': [2, 4, 8], 'depths': [3, 3, 6], 'downsampling_factors': [[2, 2], [2, 2], [2, 2]]}

In [66]:
net = instantiate(cfg)

In [67]:
from torchinfo import summary

In [68]:
summary(net, (2, 1, 56, 1024), device="cpu", depth=4)

Layer (type:depth-idx)                             Output Shape              Param #
ConvNext                                           [2, 128, 7, 128]          --
├─Conv2d: 1-1                                      [2, 16, 56, 1024]         800
├─ModuleList: 1-2                                  --                        --
│    └─ModuleList: 2-1                             --                        --
│    │    └─ConvNextBlock: 3-1                     [2, 16, 56, 1024]         --
│    │    │    └─Conv2d: 4-1                       [2, 16, 56, 1024]         800
│    │    │    └─Sequential: 4-2                   [2, 16, 56, 1024]         9,280
│    │    │    └─Identity: 4-3                     [2, 16, 56, 1024]         --
│    │    └─ModuleList: 3-2                        --                        --
│    │    │    └─ConvNextBlock: 4-4                [2, 16, 56, 1024]         10,080
│    │    │    └─ConvNextBlock: 4-5                [2, 16, 56, 1024]         10,080
│    │    │    └─ConvN